In [1]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-2.2.5/en_core_web_lg-2.2.5.tar.gz

     |████████████████████████████████| 827.9 MB 3.4 kB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-py3-none-any.whl size=829180942 sha256=1e19c580bfb3f8b250ec33a873d1468f181e4ce8c7faa75b360f901abaa4ad98
  Stored in directory: /root/.cache/pip/wheels/11/95/ba/2c36cc368c0bd339b44a791c2c1881a1fb714b78c29a4cb8f5
Successfully built en-core-web-lg


In [1]:
!pip install bert-tensorflow

     |████████████████████████████████| 64 kB 1.7 MB/s 


In [2]:
#%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!ln -s /content/gdrive/MyDrive/ /mydrive
!ls /mydrive

'2020 slides'
 2021-online-language-offline-impact-main
 An_Xception_Based_Convolutional_Neural_Network_for_Scene_Image_Classification_with_Transfer_Learning.pdf
'API key'
'Application for Lease- Form for Email (Applicant) (1).gdoc'
'Application for Lease- Form for Email (Applicant).gdoc'
'Application for Lease- Form for Email (Applicant).pdf'
'Colab Notebooks'
'CV -Xizhi Wu.pdf'
'FOW Documents'
 MyDrive
'NT15 LEASE 2021.gdoc'
'NT15 LEASE 2021.pdf'
'Review Group 9.gdoc'
 Xizhi_Wu_UnderGrad_Transcript.pdf


In [4]:
%cd gdrive/MyDrive/2021-online-language-offline-impact-main

/content/gdrive/MyDrive/2021-online-language-offline-impact-main


In [5]:
!ls

 a0529-a0611_UK         dataframe_csv_EU
 a0529-a0611_US         dataframe_csv_UK
 a0529.csv	        dataframe_result_csv
 a0530.csv	        dataframe_result_csv_EU
 a0531.csv	        dataframe_result_csv_UK
 a0601.csv	        Github
 a0602.csv	        heatmap529_UK.ipynb
 a0603.csv	        heatmap530.ipynb
 a0604.csv	        Heat_Map_Online_language.ipynb
 a0605.csv	        __MACOSX
 csv		        non_violant1.csv
 csv.zip	        non_violant2.csv
 dataframe0603_UK.csv   UK
 dataframe0604_UK.csv   violant.csv
 dataframe0605_UK.csv  '新建 Microsoft PowerPoint 演示文稿.pptx'
 dataframe_csv


In [6]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [7]:
ct_path_531 = 'UK/06_07.csv'

In [8]:
ct = pd.read_csv(ct_path_531)

In [9]:
ct.shape[1]

47

In [10]:
#columns1=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message','expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media_type', 'media_numl', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']
columns2=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message', 'expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']

In [11]:
#len(columns1)
len(columns2)

47

In [12]:
ct.columns=columns2

In [13]:
ct

,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id
0,190860494284602_2965105463526744,Facebook,2020-06-07 23:59:45,2021-06-04 20:07:25,link,George Floyd protests: Groups gather around Ba...,eastbaytimes.com,Nearly a fortnight after the killing of George...,George Floyd protests: Groups gather around Ba...,1.0,https://bayareane.ws/2AbPcLo,https://www.facebook.com/190860494284602/posts...,744,-22.000000,1.0,0,0,0,0,0,0,0,0,0,0,2,2,2,1,1,0,1,2,0,0,1862079,Lamorinda Sun,LamorindaSun,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,"Your source for community news in Lafayette, M...",False,2011-03-19 05:29:56,en,0,1862079|2965105463526744
1,231890057639083_719447318883352,Facebook,2020-06-07 23:59:37,2021-07-30 09:43:37,link,George Floyd protest news: Live news from prot...,nbcnews.com,Here are the latest updates on protests across...,“Sen. Mitt Romney (R-Utah) marched with a grou...,1.0,https://www.nbcnews.com/news/us-news/live-blog...,https://www.facebook.com/231890057639083/posts...,16068,-19.666667,1.0,4,0,1,1,0,0,0,0,0,0,55,10,6,22,4,4,5,12,0,2,8603980,The Conscious Progressive,consciousprogressive,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Progressivism for the soul.,False,2018-07-15 04:16:31,en,0,8603980|719447318883352
2,482019451816684_3334383213246946,Facebook,2020-06-07 23:59:36,2021-06-13 08:01:09,link,Protesters march along Dixie Highway to Elsmer...,wcpo.com,Hundreds marched along Dixie Highway in Northe...,Protesters peacefully march along Dixie Highwa...,2.0,https://www.wcpo.com/news/region-northern-kent...,https://www.facebook.com/groups/48201945181668...,13947,-1.900000,1.0,4,1,3,2,0,0,0,0,0,0,3,2,4,2,2,2,2,2,0,1,2560158,Northern Kentucky,consciousprogressive,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_group,US,Progressivism for the soul.,False,2018-07-15 04:16:31,en,0,2560158|3334383213246946
3,899226323523823_2963460987100336,Facebook,2020-06-07 23:59:36,2021-05-25 04:37:12,link,CANDACE OWENS on race & the riots: George Floy...,newstalkwkmq.iheart.com,Candace Owens says America's African American ...,Candace Owens says America's African American ...,1.0,http://ihe.art/9jUNXBl,https://www.facebook.com/899226323523823/posts...,80,-18.000000,1.0,0,0,0,0,0,0,0,0,0,0,1,2,1,1,1,1,1,1,0,0,3261034,News Talk WKMQ,newstalkwkmq,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,News Talk WKMQ is Tupelo's New and Talk with R...,False,2016-01-22 18:04:03,en,0,3261034|2963460987100336
4,463248983746591_3968195516585236,Facebook,2020-06-07 23:59:32,2021-07-12 12:33:30,link,Attorney General Bill Barr Sets CBS's Margaret...,thegatewaypundit.com,US Attorney General William Barr on Sunday spo...,Bill Barr sets CBS's Margaret Brennan straight...,1.0,https://www.thegatewaypundit.com/2020/06/attor...,https://www.facebook.com/463248983746591/posts...,3037,-0.933333,1.0,1,0,0,0,0,0,0,0,0,0,3,2,2,2,1,2,1,2,0,2,5470235,Conservative Newsroom,Conservative.Newsroom,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,"Conservative News and Information, and connect...",False,2013-03-10 21:52:27,en,0,5470235|3968195516585236
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18

In [14]:
ct['message'] = ct['message'].fillna('')
ct['description'] = ct['description'].fillna('')
ct['title'] = ct['title'].fillna('')

In [15]:
nlp = spacy.load('en_core_web_lg', disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"])

In [16]:
def ner(txt):
    ner = dict()

    doc = nlp(txt)
    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [17]:
def ner_inline(doc):
    ner = dict()

    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [18]:
def ner_pipe(texts):
    batch = []
    for doc in nlp.pipe(texts, batch_size=200, n_threads=16):
        batch.append(ner_inline(doc))
    return batch

In [19]:
ct['NER-msg'] = ner_pipe(ct['message'])

In [20]:
ct['NER-msg'] = ct['message'].apply(ner)

In [21]:
ct['NER-desc'] = ct['description'].apply(ner)

In [22]:
ct['NER-title'] = ct['title'].apply(ner)

In [23]:
ct[['NER-msg', 'NER-desc']].head()

,NER-msg,NER-desc
0,{},{'Minneapolis': 1}
1,"{'Utah': 1, 'Washington': 1}",{}
2,{},{'Minneapolis': 1}
3,{'America': 1},{'America': 1}
4,{'DC': 1},{'US': 1}


In [24]:
ct.to_csv('a0607.csv')

In [25]:
ct=pd.read_csv('a0607.csv',lineterminator="\n")

In [26]:
ct['NER-msg'] = ct['NER-msg'].apply(ast.literal_eval)
ct['NER-desc'] = ct['NER-desc'].apply(ast.literal_eval)
ct['NER-title'] = ct['NER-title'].apply(ast.literal_eval)

In [27]:
ct_ner = ct[((ct['NER-msg'] != {}) | (ct['NER-desc'] != {}) | (ct['NER-title'] != {}))]

In [28]:
ct_ner[['NER-msg', 'NER-desc', 'NER-title']].head()

,NER-msg,NER-desc,NER-title
0,{},{'Minneapolis': 1},{}
1,"{'Utah': 1, 'Washington': 1}",{},{}
2,{},{'Minneapolis': 1},{}
3,{'America': 1},{'America': 1},{}
4,{'DC': 1},{'US': 1},{}


In [29]:
ct_ner['NER-msg'] = ct_ner['NER-msg'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-desc'] = ct_ner['NER-desc'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-title'] = ct_ner['NER-title'].apply(lambda x: list(set(x.keys())))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [30]:
ct_ner['NER-msg']

0                                                  []
1                                  [Washington, Utah]
2                                                  []
3                                           [America]
4                                                [DC]
                             ...                     
18887                                   [Minneapolis]
18888                      [US, America, Minneapolis]
18889                           [North west Province]
18891                             [the United States]
18893    [France, America, Fort de Joux, Minneapolis]
Name: NER-msg, Length: 13321, dtype: object

In [31]:
ct_ner['NER'] = ct_ner['NER-msg'] + ct_ner['NER-desc'] + ct_ner['NER-title']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [32]:
ct_ner['NER']

0                                       [Minneapolis]
1                                  [Washington, Utah]
2                                       [Minneapolis]
3                                  [America, America]
4                                            [DC, US]
                             ...                     
18887                                   [Minneapolis]
18888     [US, America, Minneapolis, US, Minneapolis]
18889                           [North west Province]
18891          [the United States, the United States]
18893    [France, America, Fort de Joux, Minneapolis]
Name: NER, Length: 13321, dtype: object

In [33]:
ct_ner['NER'] = ct_ner['NER'].apply(set)
ct_ner['NER'] = ct_ner['NER'].apply(list)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [34]:
ct_ner['NER'] = ct_ner['NER'].apply(lambda x: [s.lower() for s in x])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [35]:
ner_lst = ct_ner['NER'].values

In [36]:
ner_lst

array([list(['minneapolis']), list(['washington', 'utah']),
       list(['minneapolis']), ..., list(['north west province']),
       list(['the united states']),
       list(['france', 'minneapolis', 'fort de joux', 'america'])],
      dtype=object)

In [37]:
m=0
list=[]
for n in ner_lst:
  for i in range(len(n)):
    list.append(ner_lst[m][i])
  m+=1

In [38]:
list

['minneapolis',
 'washington',
 'utah',
 'minneapolis',
 'america',
 'dc',
 'us',
 'marlborough',
 'berkeley',
 'houston',
 'america',
 'los angeles',
 'los angeles county',
 'minneapolis',
 'washington d.c.',
 'bloomfield',
 'nj',
 'essex county',
 'bloomfield',
 'minneapolis',
 'new york',
 'new york city',
 'north dakota',
 'north dakota',
 'us',
 'the united states',
 'dallas',
 'the united states',
 'dallas',
 'melbourne',
 'the united states',
 'dallas',
 'america',
 'new zealand',
 'lovepower',
 'china',
 'washington',
 'd.c.',
 'us',
 'minnesota',
 'memphis',
 'new zealand',
 'lovepower',
 'memphis',
 'montana',
 'minneapolis',
 'washington',
 'd.c.',
 'new york',
 'minneapolis',
 'london',
 'minneapolis',
 'london',
 'london',
 'u.k.',
 'minneapolis',
 'minneapolis',
 'washington',
 'd.c.',
 'america',
 'the united states',
 'the united states',
 'minneapolis',
 'new york city',
 'new york city',
 'nyc',
 'new york',
 'minneapolis',
 'minneapolis',
 'minneapolis',
 'kentucky',

In [39]:
c = collections.Counter()

In [40]:
dict_a=collections.Counter(list).most_common()

In [41]:
dict_a

[('minneapolis', 3466),
 ('america', 1708),
 ('us', 1615),
 ('the united states', 909),
 ('washington', 903),
 ('u.s.', 848),
 ('minnesota', 614),
 ('london', 557),
 ('houston', 520),
 ('d.c.', 514),
 ('uk', 466),
 ('new york', 451),
 ('new york city', 385),
 ('texas', 373),
 ('los angeles', 345),
 ('north carolina', 296),
 ('usa', 290),
 ('chicago', 289),
 ('california', 268),
 ('atlanta', 251),
 ('philadelphia', 216),
 ('australia', 209),
 ('nyc', 198),
 ('seattle', 193),
 ('canada', 186),
 ('santa cruz', 185),
 ('buffalo', 180),
 ('dc', 165),
 ('san francisco', 165),
 ('florida', 161),
 ('bristol', 159),
 ('denver', 158),
 ('china', 149),
 ('portland', 139),
 ('richmond', 131),
 ('alabama', 129),
 ('louisville', 122),
 ('washington dc', 120),
 ('birmingham', 118),
 ('germany', 114),
 ('connecticut', 113),
 ('britain', 109),
 ('boston', 107),
 ('raeford', 104),
 ('israel', 102),
 ('milwaukee', 95),
 ('maryland', 94),
 ('dallas', 92),
 ('south africa', 91),
 ('sydney', 89),
 ('ghana',

In [42]:
dataframe531=pd.DataFrame(dict_a)

In [43]:
len(dict_a)

3122

In [44]:
dataframe531.to_csv('dataframe0607_UK.csv')